In [101]:
!pip install gdown pandas nltk pandarallel spacy autocorrect


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [102]:
import pandas as pd
import nltk

## Download base dataset
Dataset is from https://www.kaggle.com/datasets/rtatman/blog-authorship-corpus. Contains over 700000 blog posts with metadata about author and blog

In [103]:
# Download blogtext.csv (700 000 blog posts)
!gdown 1PJbVYUmRr0_HTwGNtplnu8lG-UCDoXZJ

Downloading...
From (uriginal): https://drive.google.com/uc?id=1PJbVYUmRr0_HTwGNtplnu8lG-UCDoXZJ
From (redirected): https://drive.google.com/uc?id=1PJbVYUmRr0_HTwGNtplnu8lG-UCDoXZJ&confirm=t&uuid=c7ef063b-847a-416c-ad6a-44c3c34bbdd7
To: /Users/kristian.aars/PycharmProjects/AIAC536-NextWordPrediction/blogtext.csv
100%|████████████████████████████████████████| 800M/800M [00:23<00:00, 33.6MB/s]


## Import data

### Import from csv-file using Pandas
Specify the number of blog posts to process, as the processing can take time

In [104]:
N_BLOGPOSTS = 10000

df = pd.read_csv('blogtext.csv', nrows=N_BLOGPOSTS)

### Tokenize sentences
This splits all the sentences into arrays, and then explodes the array back to the dataframe as separate sentences. After this, all sentences which are not recognized as strings are removed.

In [105]:
df.text = df.text.transform(lambda t: nltk.sent_tokenize(t))

# Explode array into separate rows
df = df.explode('text')

# Remove all sentences not recognized as strings (numbers, lists etc.)
mask = df['text'].apply(lambda x: isinstance(x, str))
df = df[mask]

# Remove all sentences not containing anything
df = df[df['text'] != '']

## Preprocess and clean text
This step will clean the text contents and remove unwanted blog-posts from the dataset. Blogposts containing non-english words may be removed and a set percentage of stop-words can be removed to reduce their occurrence in the dataset.

### Specify cleaning parameters

In [106]:
remove_stopwords=False
rem_stopword_percent=0.00
remove_sentence_with_unknown_words=True
min_word_count=4

In [107]:
from nltk import word_tokenize
from nltk.corpus import words as en_words
from nltk.corpus import stopwords

import spacy

import re
import random

from pandarallel import pandarallel

spacy_nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))

### Define some methods for text-cleaning and filtering

In [108]:
def contains_non_lexi_word(sentence):
    sentence = re.sub(r'[^a-zA-Z\s]', '', sentence)
    lemmatized_doc = spacy_nlp(sentence)
    
    english_words = en_words.words()
    
    for token in lemmatized_doc:
        word = token.lemma_.lower()
        
        if word not in english_words:                               
            #print("Found non-english word {0}".format(word))
            return True
        
def contains_number(sentence):
    return any(char.isdigit() for char in sentence)

def word_count(text):
    words = word_tokenize(text)
    return len(words)

In [109]:
def clean_corpus(corpus_df, rm_contains_unknown_wrd=True, rm_contains_num=True, min_word_count=2, remove_stopwords=True, prob_remove_stopword=0.1):
    
    # Function which returns wether to keep a sentence in the dataset or not based on given parameters
    def keep_sentence(row):
        if word_count(row['text']) < min_word_count:
            #print("{0} is too short".format(row['text']))
            return False
        elif rm_contains_num and contains_number(row['text']):
            #print("{0} contains number".format(row['text']))
            return False
        elif rm_contains_unknown_wrd and contains_non_lexi_word(row['text']):
            #print("{0} contains unknown words".format(row['text']))
            return False
        else:
            #print("{0} is ok".format(row['text']))
            return True
    
    # Function to normalize and clean the text
    def clean_text(text):
        # Convert to lowercase for better normalization
        text = text.lower()
    
        # Remove punctuation
        text = re.sub(r'[^\w\s]', '', text)
    
        # Tokenize the words
        words = word_tokenize(text)
        
        # Remove stop words
        if remove_stopwords:
            if random.random() < prob_remove_stopword:
                words = [word for word in words if word not in stop_words]
        
        # Join the cleaned words back into a sentence
        cleaned_text = ' '.join(words)
    
        return cleaned_text

    pre_rem_size = corpus_df.shape[0]
    pandarallel.initialize()
    corpus_df['text'] = corpus_df['text'].apply(clean_text)
    corpus_df = corpus_df[corpus_df.parallel_apply(keep_sentence, axis=1)]
    sen_removed = pre_rem_size - corpus_df.shape[0]
    print("Removed {0} sentences because they did not conform to the sentence specifications".format(sen_removed))
    
    return corpus_df

## Let the cleaning begin
This might take some while. Please be patient. 5000 blog posts takes around 10 minutes on a M1 Pro MacBook Pro 14" laptop. 

In [110]:
df = clean_corpus(df, remove_stopwords=remove_stopwords, rm_contains_unknown_wrd=remove_sentence_with_unknown_words, prob_remove_stopword=rem_stopword_percent, min_word_count=min_word_count)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Removed 72313 sentences because they did not conform to the sentence specifications


## Lastly, save the file so it can be loaded for training

In [112]:
df.to_csv("blogtext_cleaned.csv")